In [28]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
from pymongo import MongoClient
from pymongo.errors import OperationFailure
from load_data_utils import *
from pprint import pprint

In [2]:
profile = pd.read_csv('./Purchase Redemption Data/user_profile_table.csv')
profile.head(3)

,user_id,sex,city,constellation
0,2,1,Shanghai,Leo
1,12,1,Guangzhou,Capricorn
2,22,1,Shanghai,Gemini


In [3]:
cash_flow = pd.read_csv('./Purchase Redemption Data/user_balance_table.csv')
cash_flow.head(3)

,user_id,report_date,tBalance,yBalance,total_purchase_amt,direct_purchase_amt,purchase_bal_amt,purchase_bank_amt,total_redeem_amt,consume_amt,transfer_amt,tftobal_amt,tftocard_amt,share_amt,category1,category2,category3,category4
0,1,20140805,20385,20383,2,0,0,0,0,0,0,0,0,2,NaN,NaN,NaN,NaN
1,1,20140808,20391,20389,2,0,0,0,0,0,0,0,0,2,NaN,NaN,NaN,NaN
2,1,20140811,20397,20395,2,0,0,0,0,0,0,0,0,2,NaN,NaN,NaN,NaN


In [4]:
server = SSHTunnelForwarder(
    "34.67.124.229",
    ssh_username="apple",
    ssh_pkey="/Users/apple/Desktop/CSSE433/project-cli-keys/key-instance-1",
    ssh_private_key_password="csse433",
    remote_bind_address=("127.0.0.1", 27017)
)
server.start()
host = "localhost"
port = server.local_bind_port
try:
    cli = MongoClient(f'mongodb://{host}:{port}/?authSource=admin')
    cli.server_info()
except OperationFailure as err:
    print(err)

In [5]:
db = cli['yuebao']
customers = db['customer']

In [6]:
user_ids = [1,2,3]
docs = [generate_customer_dict(user_id, profile, cash_flow) for user_id in user_ids]

In [8]:
customers.insert_many(docs)

In [30]:
out = customers.find_one({'user_id': 1, 'cashflows.report_date': 20140805}, 
                   {'user_id': 1, 'cashflows.$':1, 'sex':1, 'city':1, 'constellation':1, '_id':0})

In [37]:
out

{'user_id': 1,
 'sex': 1,
 'city': 'Shanghai',
 'constellation': 'Aries',
 'cashflows': [{'report_date': 20140805,
   'balance': {'tBalance': 20385, 'yBalance': 20383},
   'purchase': {'purchase_bal_amt': 0, 'purchase_bank_amt': 0, 'revenue': 2},
   'redemption': {'consumption': {'category1': -1,
     'category2': -1,
     'category3': -1,
     'category4': -1},
    'transfer': {'tftobal_amt': 0, 'tftocard_amt': 0}}}]}

In [42]:
out['cashflows'][0]

{'report_date': 20140805,
 'balance': {'tBalance': 20385, 'yBalance': 20383},
 'purchase': {'purchase_bal_amt': 0, 'purchase_bank_amt': 0, 'revenue': 2},
 'redemption': {'consumption': {'category1': -1,
   'category2': -1,
   'category3': -1,
   'category4': -1},
  'transfer': {'tftobal_amt': 0, 'tftocard_amt': 0}}}